In [ ]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover
import math

In [ ]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
data = yf.download(ticker, period="1y", interval="1h")


In [ ]:
data

In [ ]:
# Importing through a CSV that has more data which was downloaded externally.
df = pd.read_csv('../Data/AUDCAD_H1(22007-09-26 - 2024-02-02).csv', delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0)
df.index = pd.to_datetime(df.index)


In [ ]:
# csv slice by date
start_date = '2010-01-01'
end_date = '2024-01-01'
df = df.loc[start_date:end_date]

# Write out strategy here to figure out the logic

In [ ]:
class Strat(Strategy):
    def init(self):
        pass

    def next(self):
        pass

bt = Backtest(df_slice, Strat, cash=10_000)
bt.run()
bt.plot()

In [ ]:
# help(ta.cdl_pattern(name='engulfing'))
# Size 10,000 = $1000 based on 10 pips of movement = $100 and 1000 units = 1 pip movement 
help(Strategy)

In [ ]:
class Strat(Strategy):
    def init(self):
        # Calculate the 50-period EMA and 14-period ATR and check engulfing        
        self.ema = self.I(ta.ema, pd.Series(self.data.Close), length=50)
        self.atr = self.I(ta.atr, 
                          pd.Series(self.data.High), 
                          pd.Series(self.data.Low), 
                          pd.Series(self.data.Close), 
                          length=14, plot=False)
               
        self.pullback = False
        self.pullback_count = 0
        self.consolidation_high = 0
        
        
    def next(self):
        current_ema = self.ema[-1]
        
        current_close = self.data.Close[-1]
        current_open = self.data.Open[-1]
        current_high = self.data.High[-1]

        bull_body = self.data.Close[-1] > self.data.Open[-1]
        bear_body = self.data.Close[-1] < self.data.Open[-1]

        # position sizing logic
        account_balance = math.floor(self.equity)
        risk_per_trade = account_balance * 0.01
        stop_loss = self.data.Low[-1] - self.atr[-1]
        stop_in_pips = abs(current_close - stop_loss) * 100  # Calculate stop loss in pips
        
        # Adjust position size based on the risk per trade and stop loss distance
        position_size = math.floor((risk_per_trade / (stop_in_pips * 10)) * 1000)  # Adjusted for $1000 per 10 pips at 10,000 units
        take_profit = ((stop_in_pips / 100) * 1.2) + current_close

        result = position_size 



In [ ]:
bt = Backtest(data, Strat, cash=100000, margin=0.02)
stats = bt.run()
print(stats)


In [ ]:
bt.plot()

In [ ]:
stats._trades

In [ ]:

# plot own equity curve based on backtesting data for data more than 10K
equity_curve = stats._equity_curve['Equity']

plt.figure(figsize=(10, 6))
plt.plot(equity_curve, label='Equity Curve', lw=1)  # lw is line width
plt.title('Equity Curve')
plt.xlabel('Time')
plt.ylabel('Equity')
plt.legend()
plt.grid(False)
plt.show()


In [ ]:

equity_curve = stats._equity_curve['Equity'].reset_index()

fig = px.line(equity_curve, x='index', y='Equity', labels={'index': 'Time'}, title='Strategy Performance')
fig.update_layout(height=600, xaxis_title='Time', yaxis_title='Equity', legend_title='Legend')
fig.show()
